# 🚀 AI Sales Hunter: The ERP Transformation Specialist
**Google AI Agents Intensive - Capstone Project**

## **📖 Project Overview**
This AI Sales Agent is an autonomous "Sales Development Representative" (SDR) designed to hunt for high-value Enterprise Resource Planning (ERP) transformation opportunities. 

Given just a **domain name** (e.g., `rivian.com`), the agent acts as a multi-step investigator:
1.  **🕵️‍♂️ Researches** the company's industry, products, and recent news using **live Google Search**.
2.  **📶 Detects Signals** specific to ERP needs (mergers, supply chain complexity, legacy tech debt).
3.  **🎯 Identifies Stakeholders** (CIOs, CTOs, VPs of Operations) from public records.
4.  **✍️ Drafts the Pitch**, generating a hyper-personalized cold email that connects their specific pain points to an ERP transformation solution.

## **🏗️ Architecture**
This project demonstrates the following key concepts:
*   **Multi-Agent System:** Sequential coordination of 4 specialized agents.
*   **Tool Use:** Custom wrappers for Live Google Search and Web Scraping.
*   **Observability:** A custom logging layer to trace agent reasoning steps.
*   **Structured Output:** Using Pydantic to ensure data integrity between agent handoffs.

## **⚙️ Setup & Observability**

In [ ]:
import os
import sys
import logging
import json
from datetime import datetime
from kaggle_secrets import UserSecretsClient

# --- 1. Authentication ---
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Auth Warning: Ensure 'GOOGLE_API_KEY' is in secrets. Details: {e}")

# --- 2. Logging & Observability ---
# A simple logger to trace tool execution and agent thought processes
class AgentLogger:
    def __init__(self):
        self.logs = []

    def log(self, agent_name: str, event_type: str, details: str):
        entry = {
            "timestamp": datetime.now().isoformat(),
            "agent": agent_name,
            "type": event_type,
            "details": details
        }
        self.logs.append(entry)
        print(f"[{agent_name}] {event_type}: {details}")

logger = AgentLogger()

# Suppress verbose internal logs to keep our custom tracing clean
logging.getLogger("google_genai").setLevel(logging.ERROR)
logging.getLogger("google.adk").setLevel(logging.ERROR)

from google.adk.agents import Agent, SequentialAgent, LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import FunctionTool, AgentTool
from google.genai import types

# --- 3. Core Services ---
session_service = InMemorySessionService()
memory_service = InMemoryMemoryService()

retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 503],
)

print("✅ Environment, Services, and Logger Initialized.")

## **🛠️ Tool Definition: Live Search & Web Scraping**

We define wrappers for the live `google_search` tool and a website scraper. These tools are the "eyes" of our agents, allowing them to gather real-time intelligence from the web.

In [ ]:
from google.adk.tools import FunctionTool
import google_tools

# --- 1. Live Google Search ---
def search_web(query: str) -> str:
    """
    Performs a live Google Search to find information about companies, executives, or news.
    Use this for: "Who is the CIO of X?", "Recent mergers for X", "What ERP does X use?".
    """
    logger.log("Tool", "Search", f"Query: {query}")
    try:
        # Uses the environment's native google_search tool
        results = google_tools.google_search(query)
        return json.dumps(results)
    except Exception as e:
        logger.log("Tool", "Error", str(e))
        return json.dumps({"error": str(e)})

# --- 2. Website Scraper ---
def scrape_website(url: str) -> str:
    """
    Fetches the text content of a specific URL.
    Use this to read 'About Us' pages, Press Releases, or Technical Blog posts found via search.
    """
    logger.log("Tool", "Scrape", f"URL: {url}")
    try:
        # Uses the environment's native view_text_website tool
        content = google_tools.view_text_website(url)
        # Truncate extremely long pages to fit context window
        return content[:10000]
    except Exception as e:
        logger.log("Tool", "Error", str(e))
        return json.dumps({"error": str(e)})

print("✅ Research Tools Defined.")

## **🕵️‍♂️ Agent Definition: The Sales Team**

We define four specialized agents, each with a unique persona and objective. They pass data sequentially, enriching the "lead profile" at each step.

In [ ]:
from pydantic import BaseModel, Field
from typing import List

# --- Data Schemas for Structured Handoffs ---

class CompanyProfile(BaseModel):
    domain: str
    company_name: str
    industry: str
    core_products: List[str]
    key_competitors: List[str]
    recent_news_summary: str

class ERPSignals(BaseModel):
    company_name: str
    current_tech_stack_inference: str = Field(description="Best guess at current ERP (SAP, Oracle, NetSuite) based on job posts/news.")
    transformation_triggers: List[str] = Field(description="Events triggering need for change: M&A, Global Expansion, Supply Chain issues.")
    pain_point_hypothesis: str = Field(description="Why do they need to switch ERPs NOW?")

class StakeholderList(BaseModel):
    stakeholders: List[dict] = Field(description="List of Name, Title, and why they are relevant to ERP buying.")

class SalesPitch(BaseModel):
    subject_line: str
    email_body: str
    strategy_explanation: str

# --- 1. Research Agent ---
research_agent = LlmAgent(
    name="CompanyResearchAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    tools=[FunctionTool(search_web), FunctionTool(scrape_website)],
    output_schema=CompanyProfile,
    instruction="""
    You are a Corporate Research Analyst.
    Input: A Domain Name.
    Goal: Create a comprehensive profile of the company.
    1. Search for the company name, industry, and what they sell.
    2. Find their top 3 competitors.
    3. Summarize the most recent news (last 6 months).
    """
)

# --- 2. ERP Signal Hunter ---
signal_agent = LlmAgent(
    name="ERPSignalHunterAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    tools=[FunctionTool(search_web)],
    output_schema=ERPSignals,
    instruction="""
    You are an ERP Technical Consultant.
    Input: Company Profile.
    Goal: Detect the need for ERP Transformation.
    1. Search for job postings (e.g., "Hiring SAP Manager", "Oracle Developer") to infer their current legacy stack.
    2. Look for triggers: "Rapid acquisition", "Supply chain delays", "Digital transformation initiative".
    3. Formulate a hypothesis: Why is their current system failing them?
    """
)

# --- 3. Stakeholder Finder ---
stakeholder_agent = LlmAgent(
    name="StakeholderFinderAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    tools=[FunctionTool(search_web)],
    output_schema=StakeholderList,
    instruction="""
    You are an Executive Headhunter.
    Input: Company Name & ERP Signals.
    Goal: Find the Buying Committee.
    1. Search for C-Level Execs: CIO (Chief Information Officer), CTO, CFO, VP of Operations/Supply Chain.
    2. Find specific NAMES if possible using public search (LinkedIn profiles, company leadership pages).
    3. Return a list of 3 key targets.
    """
)

# --- 4. Copywriter ---
copywriter_agent = LlmAgent(
    name="SalesCopywriterAgent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    # No external tools needed, pure synthesis
    output_schema=SalesPitch,
    instruction="""
    You are a World-Class B2B Copywriter.
    Input: Company Profile, ERP Signals, Stakeholders.
    Goal: Write a cold email that gets a meeting.
    1. Pick ONE specific stakeholder from the list to address.
    2. Hook: Start with a specific observation about their company (News/Trigger).
    3. Pain: Connect that trigger to the likely failure of their current inferred tech stack.
    4. Solution: Briefly pitch a modern Cloud ERP transformation.
    5. CTA: Soft ask for interest.
    """
)

print("✅ Sales Team Agents Defined.")

## **🔁 Orchestration: The Pipeline**

We use a `SequentialAgent` to chain these specialists together. The output of one agent becomes the context for the next.

**Flow:** `Domain` -> `Research` -> `Signals` -> `Stakeholders` -> `Copywriter` -> `Final Pitch`

In [ ]:
# --- Sequential Pipeline ---
sales_pipeline = SequentialAgent(
    name="ERPSalesPipeline",
    sub_agents=[research_agent, signal_agent, stakeholder_agent, copywriter_agent],
    description="End-to-end sales workflow for ERP transformation prospecting."
)

# --- Coordinator ---
coordinator_agent = Agent(
    name="SalesCoordinator",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    tools=[AgentTool(sales_pipeline)],
    instruction="""
    You are the Sales Coordinator.
    When the user gives you a domain name, activate the 'ERPSalesPipeline'.
    Present the final results clearly.
    """
)

print("✅ Pipeline Orchestrated.")

## **🖥️ Interactive Simulation & Execution**

We define the `run_sales_mission` function to execute the pipeline and visualize the thought process using our custom logger.

In [ ]:
async def run_sales_mission(domain_name: str):
    print(f"\n{'='*60}")
    print(f"🎯 STARTING SALES MISSION FOR: {domain_name}")
    print(f"{'='*60}\n")

    # Initialize Runner
    runner = Runner(
        agent=coordinator_agent,
        app_name="ERP_Sales_Hunter",
        session_service=session_service,
        memory_service=memory_service
    )

    # Create a session
    session_id = f"mission_{domain_name}_{os.urandom(4).hex()}"
    session = await session_service.create_session(
        app_name="ERP_Sales_Hunter", 
        user_id="sales_user", 
        session_id=session_id
    )

    # Trigger the pipeline
    prompt = f"Research the company at '{domain_name}' and generate a cold email pitch for ERP transformation."
    
    async for event in runner.run_async(
        user_id="sales_user",
        session_id=session.id,
        new_message=types.Content(role="user", parts=[types.Part(text=prompt)])
    ):
        # Trace Tool Calls via Logger
        if event.get_function_calls():
            for fc in event.get_function_calls():
                logger.log("Runner", "Tool Call", f"{fc.name} (args: {fc.args})")
        
        # Visualize Agent Outputs (JSON data passing between steps)
        if event.content and event.content.parts:
            text = event.content.parts[0].text
            if "```json" in text: # It's likely a structured handoff
                print(f"\n📝 [Structured Output]:\n{text[:200]}... (truncated)\n")
            else:
                print(f"   🤖 {text}")

    print(f"\n{'='*60}\n✅ MISSION COMPLETE\n{'='*60}\n")

# --- Execute the Simulation ---
# Using Rivian as a prime example of complex manufacturing needs
await run_sales_mission("rivian.com")